In [61]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [62]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [63]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [64]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,3,17)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

,time,open,high,low,close
0,2025-03-17 00:00:00,1.08771,1.08771,1.08706,1.08706
1,2025-03-17 00:01:00,1.08706,1.08781,1.08706,1.08781
2,2025-03-17 00:02:00,1.08781,1.08781,1.08781,1.08781
3,2025-03-17 00:03:00,1.08781,1.08788,1.08766,1.08788
4,2025-03-17 00:04:00,1.08788,1.08789,1.08788,1.08789
...,...,...,...,...,...
71915,2025-05-23 23:52:00,1.13650,1.13654,1.13644,1.13650
71916,2025-05-23 23:53:00,1.13650,1.13657,1.13650,1.13655
71917,2025-05-23 23:54:00,1.13654,1.13655,1.13647,1.13649
71918,2025-05-23 23:55:00,1.13649,1.13649,1.13643,1.13644


In [65]:
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df.tail(5)

,time,open,high,low,close
295,2025-05-23 04:00:00,1.12971,1.13210,1.12951,1.13179
296,2025-05-23 08:00:00,1.13179,1.13447,1.13083,1.13420
297,2025-05-23 12:00:00,1.13420,1.13755,1.12976,1.13336
298,2025-05-23 16:00:00,1.13336,1.13598,1.13110,1.13583
299,2025-05-23 20:00:00,1.13583,1.13694,1.13519,1.13640


In [66]:
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df

,time,open,high,low,close
0,2025-03-17,1.08771,1.09297,1.08686,1.09214
1,2025-03-18,1.09218,1.09547,1.08925,1.09447
2,2025-03-19,1.09439,1.09461,1.08606,1.09025
3,2025-03-20,1.09010,1.09176,1.08147,1.08533
4,2025-03-21,1.08494,1.08613,1.07973,1.08124
5,2025-03-24,1.08127,1.08584,1.07817,1.08008
6,2025-03-25,1.08003,1.08302,1.07771,1.07913
7,2025-03-26,1.07901,1.08028,1.07435,1.07525
8,2025-03-27,1.07539,1.08212,1.07330,1.08018
9,2025-03-28,1.08011,1.08448,1.07645,1.08306


In [67]:
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci 
daily_df["pivot"] = ((daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3).shift(1)
daily_df["prange"] = (daily_df["high"] - daily_df["low"]).shift(1)
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df[["time","S1","R1"]].tail(10)


,time,S1,R1
40,2025-05-12,1.120976,1.128318
41,2025-05-13,1.106367,1.119920
42,2025-05-14,1.111088,1.119698
43,2025-05-15,1.116194,1.123933
44,2025-05-16,1.116939,1.121615
45,2025-05-19,1.113813,1.120574
46,2025-05-20,1.118714,1.127806
47,2025-05-21,1.123691,1.128856
48,2025-05-22,1.129282,1.135631
49,2025-05-23,1.125914,1.132713


In [68]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
#ohlc4
hourly_df['ohlc4'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4

# sma 1
hourly_df['sma_1'] = hourly_df['ohlc4'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['ohlc4'].rolling(second_sma_period).mean()
hourly_df

,time,open,high,low,close,ohlc4,sma_1,prev_sma_1,sma_14
0,2025-03-17 00:00:00,1.08771,1.08855,1.08706,1.08795,1.087818,1.087818,NaN,NaN
1,2025-03-17 04:00:00,1.08795,1.08940,1.08742,1.08822,1.088247,1.088247,1.087818,NaN
2,2025-03-17 08:00:00,1.08822,1.08869,1.08686,1.08838,1.088037,1.088037,1.088247,NaN
3,2025-03-17 12:00:00,1.08838,1.09119,1.08771,1.09018,1.089365,1.089365,1.088037,NaN
4,2025-03-17 16:00:00,1.09018,1.09297,1.08972,1.09212,1.091248,1.091248,1.089365,NaN
...,...,...,...,...,...,...,...,...,...
295,2025-05-23 04:00:00,1.12971,1.13210,1.12951,1.13179,1.130777,1.130777,1.128805,1.131208
296,2025-05-23 08:00:00,1.13179,1.13447,1.13083,1.13420,1.132823,1.132823,1.130777,1.131483
297,2025-05-23 12:00:00,1.13420,1.13755,1.12976,1.13336,1.133717,1.133717,1.132823,1.131657
298,2025-05-23 16:00:00,1.13336,1.13598,1.13110,1.13583,1.134068,1.134068,1.133717,1.131738


In [69]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
#ohlc4
daily_df['ohlc4'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
# sma 11
daily_df['sma_11'] = daily_df['ohlc4'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['ohlc4'].rolling(fourth_sma_period).mean()
daily_df[["time","sma_11","sma_114","prev_sma_11"]]

,time,sma_11,sma_114,prev_sma_11
0,2025-03-17,1.089920,NaN,NaN
1,2025-03-18,1.092842,NaN,1.089920
2,2025-03-19,1.091328,NaN,1.092842
3,2025-03-20,1.087165,NaN,1.091328
4,2025-03-21,1.083010,NaN,1.087165
5,2025-03-24,1.081340,NaN,1.083010
6,2025-03-25,1.079973,NaN,1.081340
7,2025-03-26,1.077222,NaN,1.079973
8,2025-03-27,1.077748,NaN,1.077222
9,2025-03-28,1.081025,NaN,1.077748


In [70]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('17:00:00', '%H:%M:%S').time()
end_trading_section = datetime.strptime('23:45:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
#trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]
# open one trade a day
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
# Close all open positions at 11:45 PM
open_positions = daily_df[daily_df['time'].dt.time == end_trading_section]

# function to trading
daily_df["sell_stope_loss"] = (daily_df["R2"] - daily_df["S1"]) /2 + daily_df["R2"] 
daily_df["buy_stope_loss"] = (daily_df["S2"] - ((daily_df["R1"] - daily_df["S2"]) /2))
daily_df['sell_profit'] = daily_df['low'] <= daily_df["S1"]
daily_df['buy_profit'] = daily_df["high"] >= daily_df["R1"]
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,S2,S3,ohlc4,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit
0,2025-03-17,1.08771,1.09297,1.08686,1.09214,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.089920,1.089920,NaN,NaN,NaN,NaN,False,False
1,2025-03-18,1.09218,1.09547,1.08925,1.09447,1.090657,0.00611,1.092991,1.094433,1.096767,...,1.086881,1.084547,1.092842,1.092842,1.089920,NaN,1.097488,1.083826,False,True
2,2025-03-19,1.09439,1.09461,1.08606,1.09025,1.093063,0.00622,1.095439,1.096907,1.099283,...,1.089219,1.086843,1.091328,1.091328,1.092842,NaN,1.100017,1.086109,True,False
3,2025-03-20,1.09010,1.09176,1.08147,1.08533,1.090307,0.00855,1.093573,1.095591,1.098857,...,1.085023,1.081757,1.087165,1.087165,1.091328,NaN,1.099866,1.080748,True,False
4,2025-03-21,1.08494,1.08613,1.07973,1.08124,1.086187,0.01029,1.090117,1.092546,1.096477,...,1.079827,1.075897,1.083010,1.083010,1.087165,NaN,1.097691,1.074682,True,False
5,2025-03-24,1.08127,1.08584,1.07817,1.08008,1.082367,0.00640,1.084811,1.086322,1.088767,...,1.078411,1.075967,1.081340,1.081340,1.083010,NaN,1.089522,1.075211,True,True
6,2025-03-25,1.08003,1.08302,1.07771,1.07913,1.081363,0.00767,1.084293,1.086103,1.089033,...,1.076623,1.073693,1.079973,1.079973,1.081340,NaN,1.089938,1.072788,True,False
7,2025-03-26,1.07901,1.08028,1.07435,1.07525,1.079953,0.00531,1.081982,1.083235,1.085263,...,1.076672,1.074643,1.077222,1.077222,1.079973,NaN,1.085890,1.074017,True,False
8,2025-03-27,1.07539,1.08212,1.07330,1.08018,1.076627,0.00593,1.078892,1.080291,1.082557,...,1.072962,1.070697,1.077748,1.077748,1.077222,NaN,1.083256,1.069997,True,True
9,2025-03-28,1.08011,1.08448,1.07645,1.08306,1.078533,0.00882,1.081903,1.083984,1.087353,...,1.073083,1.069713,1.081025,1.081025,1.077748,NaN,1.088394,1.068673,False,True


In [71]:
def entring_stoploss_profit(row, ohlc_df) :
    if row["prev_sma_11"] > row['sma_114'] and row["low_x"] <= row["S1"] :
        return "buy"
    elif row['prev_sma_11'] < row['sma_114'] and row["high_x"] >= row["R1"]:
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df = pd.merge_asof(ohlc_df, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
18863,sell,2025-04-03 02:43:00
21576,buy,2025-04-07 00:00:00
31188,buy,2025-04-15 16:20:00
38865,buy,2025-04-23 00:28:00
42111,buy,2025-04-25 06:36:00
46319,buy,2025-04-30 04:52:00
49467,sell,2025-05-02 09:22:00
51267,sell,2025-05-05 15:25:00
52789,sell,2025-05-06 16:48:00
60530,sell,2025-05-14 01:58:00


In [72]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "daily_low",
    "high_y": "daily_high",
    "open_y": "daily_open",
    "close_y": "daily_close"
})


In [73]:
print(merged_df.columns.tolist())
merged_df


['time', 'open', 'high', 'low', 'close', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'pivot', 'prange', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', 'ohlc4', 'sma_11', 'prev_sma_11', 'sma_114', 'sell_stope_loss', 'buy_stope_loss', 'sell_profit', 'buy_profit', 'signal', 'is_first_signal']


,time,open,high,low,close,daily_open,daily_high,daily_low,daily_close,pivot,...,ohlc4,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit,signal,is_first_signal
0,2025-03-17 00:00:00,1.08771,1.08771,1.08706,1.08706,1.08771,1.09297,1.08686,1.09214,NaN,...,1.089920,1.089920,NaN,NaN,NaN,NaN,False,False,None,NaN
1,2025-03-17 00:01:00,1.08706,1.08781,1.08706,1.08781,1.08771,1.09297,1.08686,1.09214,NaN,...,1.089920,1.089920,NaN,NaN,NaN,NaN,False,False,None,NaN
2,2025-03-17 00:02:00,1.08781,1.08781,1.08781,1.08781,1.08771,1.09297,1.08686,1.09214,NaN,...,1.089920,1.089920,NaN,NaN,NaN,NaN,False,False,None,NaN
3,2025-03-17 00:03:00,1.08781,1.08788,1.08766,1.08788,1.08771,1.09297,1.08686,1.09214,NaN,...,1.089920,1.089920,NaN,NaN,NaN,NaN,False,False,None,NaN
4,2025-03-17 00:04:00,1.08788,1.08789,1.08788,1.08789,1.08771,1.09297,1.08686,1.09214,NaN,...,1.089920,1.089920,NaN,NaN,NaN,NaN,False,False,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71915,2025-05-23 23:52:00,1.13650,1.13654,1.13644,1.13650,1.12788,1.13755,1.12763,1.13640,1.129313,...,1.132365,1.132365,1.13011,1.124539,1.139264,1.119363,False,True,None,NaN
71916,2025-05-23 23:53:00,1.13650,1.13657,1.13650,1.13655,1.12788,1.13755,1.12763,1.13640,1.129313,...,1.132365,1.132365,1.13011,1.124539,1.139264,1.119363,False,True,None,NaN
71917,2025-05-23 23:54:00,1.13654,1.13655,1.13647,1.13649,1.12788,1.13755,1.12763,1.13640,1.129313,...,1.132365,1.132365,1.13011,1.124539,1.139264,1.119363,False,True,None,NaN
71918,2025-05-23 23:55:00,1.13649,1.13649,1.13643,1.13644,1.12788,1.13755,1.12763,1.13640,1.129313,...,1.132365,1.132365,1.13011,1.124539,1.139264,1.119363,False,True,None,NaN


In [74]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    if 'signal' not in data:
        print("Warning: 'signal' column is missing!")
        return
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = ((data["R1"] - data["S1"]) /2) # example SL
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = ((data["R1"] - data["S1"]) /2)
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and ((data["low"] <= data["S1"] - ((data["R1"] - data["S1"]) /2))):
                                            #or (data["high"] >= data["R1"])):
                                            #or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and ((data["high"] >= (data["R1"] - data["S1"]) /2 + data["R1"])):
                                            #or (data["low"] <= data["S1"])):
                                            #or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)

In [75]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,sell,36383.614275,2025-04-03 02:43:00,1.09071,2025-04-03 09:34:00,1.09505,0,0,{},-157.90,-2.546853,-160.446853,-160.446853,9839.553147
1,closed,EURUSD,buy,28578.614052,2025-04-07 00:00:00,1.08936,2025-04-23 00:58:00,1.13677,0,0,{},1354.91,-2.000503,1352.909497,1192.462644,11192.462644
2,closed,EURUSD,buy,59226.268034,2025-04-25 06:36:00,1.13344,2025-04-30 17:06:00,1.13565,0,0,{},130.89,-4.145839,126.744161,1319.206805,11319.206805
3,closed,EURUSD,sell,69622.368274,2025-05-02 09:22:00,1.13255,2025-05-02 16:27:00,1.13527,0,0,{},-189.37,-4.873566,-194.243566,1124.963239,11124.963239
4,closed,EURUSD,sell,49068.435747,2025-05-05 15:25:00,1.13575,2025-05-06 22:44:00,1.13772,0,0,{},-96.66,-3.434791,-100.094791,1024.868449,11024.868449
5,closed,EURUSD,sell,46456.096666,2025-05-14 01:58:00,1.1196,2025-05-14 11:07:00,1.1234,0,0,{},-176.53,-3.251927,-179.781927,845.086522,10845.086522
6,closed,EURUSD,sell,85549.053827,2025-05-16 08:49:00,1.12137,2025-05-19 10:34:00,1.12355,0,0,{},-186.50,-5.988434,-192.488434,652.598088,10652.598088
7,closed,EURUSD,sell,43996.656254,2025-05-20 21:13:00,1.12769,2025-05-21 05:09:00,1.13124,0,0,{},-156.19,-3.079766,-159.269766,493.328323,10493.328323
8,closed,EURUSD,buy,63003.63531,2025-05-22 12:06:00,1.12971,2025-05-22 20:19:00,1.12653,0,0,{},-200.35,-4.410254,-204.760254,288.568068,10288.568068


In [76]:
mm = bt.plot_balance()
display(mm)

In [77]:
pnl_chart = bt.plot_pnl()
display(pnl_chart)

In [78]:
evaluate_backtest(bt.trades)

biggest_profit: 1354.91
daily_drawdown: -200.35
max_drawown: -1030.64
avg_win: 742.9
avg_loss: -166.21
count_profit_trades: 2
count_loss_trades: 7
Win Rate1: 22.22%
win_rate 0.29
rrr: 4.47


,order_type,profit
0,buy,1285.45
1,sell,-963.15


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,current_max,drawdown,dayofweek,month,year
0,closed,EURUSD,sell,36383.614275,2025-04-03 02:43:00,1.09071,2025-04-03 09:34:00,1.09505,0,0,...,-157.90,-2.546853,-160.446853,-160.446853,9839.553147,-160.446853,0.0,3,2025-04,2025
1,closed,EURUSD,buy,28578.614052,2025-04-07 00:00:00,1.08936,2025-04-23 00:58:00,1.13677,0,0,...,1354.91,-2.000503,1352.909497,1192.462644,11192.462644,1192.462644,0.0,2,2025-04,2025
2,closed,EURUSD,buy,59226.268034,2025-04-25 06:36:00,1.13344,2025-04-30 17:06:00,1.13565,0,0,...,130.89,-4.145839,126.744161,1319.206805,11319.206805,1319.206805,0.0,2,2025-04,2025
3,closed,EURUSD,sell,69622.368274,2025-05-02 09:22:00,1.13255,2025-05-02 16:27:00,1.13527,0,0,...,-189.37,-4.873566,-194.243566,1124.963239,11124.963239,1319.206805,-194.243566,4,2025-05,2025
4,closed,EURUSD,sell,49068.435747,2025-05-05 15:25:00,1.13575,2025-05-06 22:44:00,1.13772,0,0,...,-96.66,-3.434791,-100.094791,1024.868449,11024.868449,1319.206805,-294.338356,1,2025-05,2025
5,closed,EURUSD,sell,46456.096666,2025-05-14 01:58:00,1.1196,2025-05-14 11:07:00,1.1234,0,0,...,-176.53,-3.251927,-179.781927,845.086522,10845.086522,1319.206805,-474.120283,2,2025-05,2025
6,closed,EURUSD,sell,85549.053827,2025-05-16 08:49:00,1.12137,2025-05-19 10:34:00,1.12355,0,0,...,-186.50,-5.988434,-192.488434,652.598088,10652.598088,1319.206805,-666.608717,0,2025-05,2025
7,closed,EURUSD,sell,43996.656254,2025-05-20 21:13:00,1.12769,2025-05-21 05:09:00,1.13124,0,0,...,-156.19,-3.079766,-159.269766,493.328323,10493.328323,1319.206805,-825.878483,2,2025-05,2025
8,closed,EURUSD,buy,63003.63531,2025-05-22 12:06:00,1.12971,2025-05-22 20:19:00,1.12653,0,0,...,-200.35,-4.410254,-204.760254,288.568068,10288.568068,1319.206805,-1030.638737,3,2025-05,2025


ValueError: Plotly Express cannot process wide-form data with columns of different type.

In [ ]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                        open = ohlc_df['open'],
                                        high = ohlc_df['high'],
                                        low = ohlc_df['low'],
                                        close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["sell_stope_loss", "R2", "buy_stope_loss", "S2"]
colors = {'sell_stope_loss': 'red', 'buy_stope_loss': 'red', 'R2': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig